<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Configure-Azure-ML-workspace" data-toc-modified-id="Configure-Azure-ML-workspace-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configure Azure ML workspace</a></span></li><li><span><a href="#HyperDrive-Pipeline" data-toc-modified-id="HyperDrive-Pipeline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>HyperDrive Pipeline</a></span><ul class="toc-item"><li><span><a href="#Create-Resources-for-Training-Experiments" data-toc-modified-id="Create-Resources-for-Training-Experiments-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create Resources for Training Experiments</a></span></li><li><span><a href="#Hyperparameter-Tunning" data-toc-modified-id="Hyperparameter-Tunning-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Hyperparameter Tunning</a></span></li></ul></li></ul></div>

# Introduction

# Configure Azure ML workspace

For this project we will be using an Azure Machine Learning Notebook VM, therefore we can skip setting up the environment.

To start we need to initialize our workspace and create a Azule ML experiment.

In [ ]:
from azureml.core import Workspace, Experiment

# Initialize a workspace object for an existing Azure Machine Learning Workspace
ws = Workspace.get(name="udacity-project")

# Create a experiment
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

# HyperDrive Pipeline

## Create Resources for Training Experiments

Now that we have initialized our workspace and created our experiment, it is time to define our resources.

In this section you will create default compute clusters for use by the notebook and any other necessary operations we need.

In order to create a cluster we need to specify a compute configuration that defines the `type of machine` to be used and the `scalability behaviors`. Also, it is necessary to define the name of the cluster which must be unique within the workspace. This name is used to address the cluster later.

For this project we use a CPU cluster with following parameters:

* `type of the machine`:

    * `vm_size`: Defines the size of the virtual machine. We use here "STANDARD_D2_V2" (more details [here](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs#dv2-series))

* `Scalability behaviors`:

    * `min_nodes`: Sets minimun size of the cluster. Setting the minimum to 0 the cluster will shut down all nodes while not in use. If you use another value you are able to have faster start-up times, but you will also be billed when the cluster is not in use.

    * `max_nodes`: Sets the maximun size of the cluster. Larger number allows for more concurrency and a greater distributed processing of scale-out jobs.



In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Define CPU cluster name
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0, # when innactive
                                                           max_nodes=4) # when busy

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# from azureml.core.compute import ComputeTarget, AmlCompute

# # TODO: Create compute cluster
# # Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# # max_nodes should be no greater than 4.

# # name of CPU cluster
# amlcompute_cluster_name = "cpu-cluster"
# vm_size = 'STANDARD_D2_V2'
# max_nodes = 4

# compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,max_nodes=max_nodes)
# aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

## Hyperparameter Tunning

In this example using HyperDrive we use [`random sampling`](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py) to try different configuration sets of hyperparameters to maximize the chosen primary metric, accuracy. The function `choice` specify a discrete set of options to sample from.

The hyperparameters and metric used are defined in the script `train.py`.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0)
    '--max_iter': choice(range(10,110,10))
    }
)


# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# policy = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#early-termination

# options

# policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5) # try this

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###

# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py

# Create a SKLearn estimator for use with train.py

est = SKLearn( 
    source_directory='.',
    compute_target=cpu_cluster,
    vm_size=vm_size, 
    vm_priority='lowpriority'
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###